In [1]:
import pprint
import pandas as pd

In [2]:
import sqlite3
from sqlite3 import Error

In [3]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()


create_connection(r"./db/chinook.db")

2.6.0


In [9]:
conn = sqlite3.connect(r"./db/chinook.db")

In [10]:
cur = conn.cursor()

In [11]:
pp = pprint.PrettyPrinter(indent=4)

def sql_cmd(cmd_string):
    """Helper function to return sql query as a string"""
    result = cur.execute(cmd_string)
    return result

def query_pp(response):
    """Pretty print the response with the column names"""
    columns = [x[0] for x in [*data.description]]
    res = data.fetchall()
        
    df = pd.DataFrame(data = res, columns = columns)
    print(df)

In [12]:
data = sql_cmd("""
SELECT 
  art.name as Artist,
  t.Name as Track, 
  g.name as Genre, 
  ROUND(MAX(milliseconds)/60000.0, 2) as Length_in_minutes,
  a.title as Album_title
FROM
  tracks t
INNER JOIN genres g
ON t.GenreID = g.GenreId
INNER JOIN albums a
ON a.AlbumId = t.AlbumID
INNER JOIN artists art
ON art.ArtistId = a.artistid
GROUP BY
  t.GenreId
""")

query_pp(data)



                                              Artist  \
0                                       Led Zeppelin   
1                                        Miles Davis   
2                                        Iron Maiden   
3                                          Green Day   
4                                           BackBeat   
5                                          Buddy Guy   
6                                      Legião Urbana   
7                                      Lenny Kravitz   
8                                      Amy Winehouse   
9                                         Passengers   
10                               Toquinho & Vinícius   
11                                     Frank Sinatra   
12                                       Iron Maiden   
13                                     Amy Winehouse   
14                                        Jamiroquai   
15                            Habib Koité and Bamada   
16                                       Planet 

In [13]:
query_pp(sql_cmd(
"""
Select firstname || " " || lastname as Employee_name, title, MIN(HireDate) as Hired
FROM
  employees e
GROUP BY title;"""
))

      Employee_name                Title                Hired
0      Andrew Adams      General Manager  2002-08-14 00:00:00
1  Michael Mitchell           IT Manager  2003-10-17 00:00:00
2       Robert King             IT Staff  2004-01-02 00:00:00
3     Nancy Edwards        Sales Manager  2002-05-01 00:00:00
4      Jane Peacock  Sales Support Agent  2002-04-01 00:00:00


In [14]:
query_pp(sql_cmd(
"""
SELECT 
  c.CustomerID, 
  c.firstname || " " || c.lastname as customer,
  e.firstname || " " || e.lastname as employee,
  e.title
FROM customers c, employees e
WHERE c.SupportRepid = e.employeeid
  AND e.title = "Sales Support Agent"
"""))

    CustomerId               customer       employee                Title
0            1         Luís Gonçalves   Jane Peacock  Sales Support Agent
1            3      François Tremblay   Jane Peacock  Sales Support Agent
2           12        Roberto Almeida   Jane Peacock  Sales Support Agent
3           15      Jennifer Peterson   Jane Peacock  Sales Support Agent
4           18        Michelle Brooks   Jane Peacock  Sales Support Agent
5           19              Tim Goyer   Jane Peacock  Sales Support Agent
6           24          Frank Ralston   Jane Peacock  Sales Support Agent
7           29           Robert Brown   Jane Peacock  Sales Support Agent
8           30         Edward Francis   Jane Peacock  Sales Support Agent
9           33         Ellie Sullivan   Jane Peacock  Sales Support Agent
10          37        Fynn Zimmermann   Jane Peacock  Sales Support Agent
11          38        Niklas Schröder   Jane Peacock  Sales Support Agent
12          42           Wyatt Girard 

In [133]:
query_pp(sql_cmd("""
SELECT firstname || ' ' || lastname as user_name, SUM(i_i.quantity) as tracks_purchased
FROM customers c
INNER JOIN invoices i
ON c.customerid = i.customerid
INNER JOIN invoice_items i_i
ON i.invoiceid = i_i.invoiceid
GROUP BY c.customerid
"""
))

                user_name  tracks_purchased
0          Luís Gonçalves                38
1           Leonie Köhler                38
2       François Tremblay                38
3            Bjørn Hansen                38
4   František Wichterlová                38
5             Helena Holý                38
6           Astrid Gruber                38
7            Daan Peeters                38
8            Kara Nielsen                38
9         Eduardo Martins                38
10        Alexandre Rocha                38
11        Roberto Almeida                38
12         Fernanda Ramos                38
13           Mark Philips                38
14      Jennifer Peterson                38
15           Frank Harris                38
16             Jack Smith                38
17        Michelle Brooks                38
18              Tim Goyer                38
19             Dan Miller                38
20            Kathy Chase                38
21        Heather Leacock       